# 3. Merging 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
import pickle
from sklearn.metrics import fbeta_score
import winsound
from bs4 import BeautifulSoup
import time

In [3]:
# Make master fight_totals dataframe

fight_totals_files = os.listdir('data/ufc_stats/fight_totals2')
sig_strike_files = os.listdir('data/ufc_stats/sig_strikes2')


In [4]:
# Make master fight_totals dataframe
fight_totals = pd.DataFrame()

for file in fight_totals_files: 
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/fight_totals2/'+file)
    # add fight id column
    df['fight_id'] = file2[:-7]
    fight_totals = fight_totals.append(df)
fight_totals = fight_totals.reset_index(drop=True)

fight_totals

,Unnamed: 0,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,...,B_Ctrl_time_min,B_Ctrl_time_sec,A_Ctrl_time_tot,B_Ctrl_time_tot,details,event_title,event_url,date,Winner,fight_id
0,0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,...,0,3.0,316.0,3.0,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,"October 03, 2020","Holly Holm ""The Preacher's Daughter""",0005e00b07cee542
1,0,Joshua Burkman,Josh Neer,1,0,35,88,19,39,0.39,...,3,10.0,405.0,190.0,Abe Belardo ...,UFC 61: Bitter Rivals,http://www.ufcstats.com/event-details/f70144ca...,"July 08, 2006","Joshua Burkman ""The People's Warrior""",000da3152b7b5ab1
2,0,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,...,2,43.0,53.0,163.0,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,"March 19, 2022","Paddy Pimblett ""The Baddy""",001441f70c293931
3,0,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,...,0,0.0,0.0,0.0,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,"October 18, 2019","Ben Sosoli ""Combat Wombat""",0019ec81fd706ade
4,0,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,...,0,0.0,319.0,0.0,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,"March 14, 2015",Jared Rosholt,0027e179b743c86c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8383,0,Rodrigo Gracie,Daiju Takase,0,0,39,65,10,27,0.60,...,0,NaN,NaN,NaN,NaN,PRIDE Bushido 1,http://www.ufcstats.com/event-details/cb46dfac...,"October 05, 2003",Rodrigo Gracie,ffdfa7ba590eddaa
8384,0,Tim Sylvia,Wesley Correira,0,0,138,214,26,65,0.64,...,0,0.0,3.0,0.0,to Corner Stoppage,UFC 39: The Warriors Return,http://www.ufcstats.com/event-details/0cf93551...,"September 27, 2002","Tim Sylvia ""The Maine-iac""",ffe4379d6bd1e82b
8385,0,Kimo Leopoldo,David Abbott,0,0,1,1,0,0,1.00,...,0,1.0,112.0,1.0,Arm Triangle From Mount,UFC 43: Meltdown,http://www.ufcstats.com/event-details/108afe61...,"June 06, 2003",Kimo Leopoldo,ffe629a5232a878b
8386,0,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,...,0,0.0,45.0,0.0,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,"February 28, 2015","Tony Ferguson ""El Cucuy""",ffea776913451b6d


In [18]:
sig_strike_agg = pd.DataFrame()
for file in sig_strike_files:
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/sig_strikes2/'+file)
    df['fight_id'] = file2[:-11]
    sig_strike_agg = sig_strike_agg.append(df)
sig_strike_agg = sig_strike_agg.reset_index(drop=True)

sig_strike_agg

,Unnamed: 0,Fighter_A,Fighter_B,A_Head_Strikes_land,A_Head_Strikes_att,B_Head_Strikes_land,B_Head_Strikes_att,A_Head_Strikes_percent,B_Head_Strikes_percent,A_Body_Strikes_land,...,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent,details,event_title,event_url,fight_id
0,0,Holly Holm,Irene Aldana,81,214,38,128,0.378505,0.296875,56,...,13,28,0,0,0.464286,NaN,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,0005e00b07cee542
1,0,Joshua Burkman,Josh Neer,22,72,17,36,0.305556,0.472222,9,...,5,10,7,9,0.500000,0.777778,Abe Belardo ...,UFC 61: Bitter Rivals,http://www.ufcstats.com/event-details/f70144ca...,000da3152b7b5ab1
2,0,Paddy Pimblett,Kazula Vargas,1,4,5,7,0.250000,0.714286,0,...,1,1,3,3,1.000000,1.000000,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,001441f70c293931
3,0,Greg Hardy,Ben Sosoli,22,72,15,78,0.305556,0.192308,5,...,0,0,0,0,NaN,NaN,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,0019ec81fd706ade
4,0,Jared Rosholt,Josh Copeland,19,41,7,34,0.463415,0.205882,3,...,10,17,0,0,0.588235,NaN,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,0027e179b743c86c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8383,0,Rodrigo Gracie,Daiju Takase,25,50,6,17,0.500000,0.352941,12,...,23,37,3,3,0.621622,1.000000,NaN,PRIDE Bushido 1,http://www.ufcstats.com/event-details/cb46dfac...,ffdfa7ba590eddaa
8384,0,Tim Sylvia,Wesley Correira,115,186,21,60,0.618280,0.350000,13,...,0,0,0,0,NaN,NaN,to Corner Stoppage,UFC 39: The Warriors Return,http://www.ufcstats.com/event-details/0cf93551...,ffe4379d6bd1e82b
8385,0,Kimo Leopoldo,David Abbott,0,0,0,0,NaN,NaN,1,...,0,0,0,0,NaN,NaN,Arm Triangle From Mount,UFC 43: Meltdown,http://www.ufcstats.com/event-details/108afe61...,ffe629a5232a878b
8386,0,Tony Ferguson,Gleison Tibau,5,27,2,18,0.185185,0.111111,3,...,3,8,0,0,0.375000,NaN,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,ffea776913451b6d


In [19]:
fight_totals.to_csv('data/final/aggregates/All_Fight_Totals.csv', index=False)
sig_strike_agg.to_csv('data/final/aggregates/All_Sig_Strikes.csv', index=False)

In [20]:
# get the columns that are in both dataframes
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['fight_id', 'Fighter_B', 'details', 'event_url', 'event_title', 'Fighter_A']

In [ ]:
# drop unnamed
fight_totals = fight_totals.drop(columns=['Unnamed: 0'])
sig_strike_agg = sig_strike_agg.drop(columns=['Unnamed: 0'])

In [22]:
# drop details from sig_strike_agg
sig_strike_agg = sig_strike_agg.drop(columns=['details'])

In [23]:
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['fight_id', 'Fighter_B', 'event_url', 'event_title', 'Fighter_A']

In [24]:
# merge the two dataframes, using the common columns as the key
fight_DF = pd.merge(fight_totals, sig_strike_agg, on=common_cols, how= 'inner')
fight_DF

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,B_Clinch_Strikes_land,B_Clinch_Strikes_att,A_Clinch_Strikes_percent,B_Clinch_Strikes_percent,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent
0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,0.37,...,4,4,1.000000,1.000000,13,28,0,0,0.464286,NaN
1,Joshua Burkman,Josh Neer,1,0,35,88,19,39,0.39,0.48,...,5,9,0.571429,0.555556,5,10,7,9,0.500000,0.777778
2,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,0.77,...,2,2,0.000000,1.000000,1,1,3,3,1.000000,1.000000
3,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,0.26,...,0,1,1.000000,0.000000,0,0,0,0,NaN,NaN
4,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,0.25,...,3,8,0.750000,0.375000,10,17,0,0,0.588235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8381,Rodrigo Gracie,Daiju Takase,0,0,39,65,10,27,0.60,0.37,...,6,14,0.650000,0.428571,23,37,3,3,0.621622,1.000000
8382,Tim Sylvia,Wesley Correira,0,0,138,214,26,65,0.64,0.40,...,2,3,0.796610,0.666667,0,0,0,0,NaN,NaN
8383,Kimo Leopoldo,David Abbott,0,0,1,1,0,0,1.00,0.00,...,0,0,NaN,NaN,0,0,0,0,NaN,NaN
8384,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,0.10,...,0,0,NaN,NaN,3,8,0,0,0.375000,NaN


In [25]:
# check for nulls
fight_DF.isnull().sum()

Fighter_A                      0
Fighter_B                     20
A_Kd                           0
B_Kd                           0
A_Sig_strike_land              0
                            ... 
A_Ground_Strikes_att           0
B_Ground_Strikes_land          0
B_Ground_Strikes_att           0
A_Ground_Strikes_percent    2646
B_Ground_Strikes_percent    3795
Length: 79, dtype: int64

In [26]:
# drop when Fighter_B is null
fight_DF = fight_DF.dropna(subset=['Fighter_B'])

In [27]:
missing = fight_DF.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing

A_Total_Strikes_percent         37
Winner                          57
B_Total_Strikes_percent         65
A_Head_Strikes_percent          96
B_Distance_Strikes_percent     169
A_Distance_Strikes_percent     172
B_Head_Strikes_percent         182
details                        282
A_Body_Strikes_percent        1013
A_Ctrl_time_sec               1211
B_Ctrl_time_sec               1211
A_Ctrl_time_tot               1211
B_Ctrl_time_tot               1211
B_Body_Strikes_percent        1265
A_Leg_Strikes_percent         1814
B_Leg_Strikes_percent         1857
A_Clinch_Strikes_percent      2027
B_Clinch_Strikes_percent      2111
A_Ground_Strikes_percent      2639
A_Takedown_percent            2687
B_Takedown_percent            3184
B_Ground_Strikes_percent      3788
A_Sub_Attempts_att            8366
B_Sub_Attempts_att            8366
dtype: int64

In [19]:
# drop when Winner is missing
fight_DF = fight_DF.dropna(subset=['Winner'])

In [20]:
# change all the nulls to 0
fight_DF = fight_DF.fillna(0)

In [21]:
fight_DF.to_csv('data/final/aggregates/Fight_DF.csv', index=False)

In [6]:
fight_DF = pd.read_csv('data/final/aggregates/Fight_DF.csv')
fight_DF

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,B_Clinch_Strikes_land,B_Clinch_Strikes_att,A_Clinch_Strikes_percent,B_Clinch_Strikes_percent,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent
0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,0.37,...,4,4,1.000000,1.000000,13,28,0,0,0.464286,0.000000
1,Joshua Burkman,Josh Neer,1,0,35,88,19,39,0.39,0.48,...,5,9,0.571429,0.555556,5,10,7,9,0.500000,0.777778
2,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,0.77,...,2,2,0.000000,1.000000,1,1,3,3,1.000000,1.000000
3,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,0.26,...,0,1,1.000000,0.000000,0,0,0,0,0.000000,0.000000
4,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,0.25,...,3,8,0.750000,0.375000,10,17,0,0,0.588235,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8304,Rodrigo Gracie,Daiju Takase,0,0,39,65,10,27,0.60,0.37,...,6,14,0.650000,0.428571,23,37,3,3,0.621622,1.000000
8305,Tim Sylvia,Wesley Correira,0,0,138,214,26,65,0.64,0.40,...,2,3,0.796610,0.666667,0,0,0,0,0.000000,0.000000
8306,Kimo Leopoldo,David Abbott,0,0,1,1,0,0,1.00,0.00,...,0,0,0.000000,0.000000,0,0,0,0,0.000000,0.000000
8307,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,0.10,...,0,0,0.000000,0.000000,3,8,0,0,0.375000,0.000000


# Updating fight_DF with New Fights

Assuming by now/here, the new fights have been downloaded. 

In this case, just need to:

1. Identify previous 5 UFC fights
2. Check versus downloaded fights


In [ ]:
def update_fight_DF()

    # get the fights that need to be updated from the previous notebook (scraping)
    # TODO: Complete this function